### How to extract data from spotify (credit to Sejung Jenn via Kaggle)

In [1]:
#!pip install python-dotenv spotipy
#!pip install spotipy
import csv
import os
import re
from dotenv import load_dotenv
import pandas as pd
import spotipy as spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import base64
from requests import post 
import json 

In [2]:
load_dotenv()

client_id = os.getenv("CLIENT_ID", "3afae00bb43841688a4d44a9240a1ead") # tyoe in your client_id in between the quotation marks
client_secret = os.getenv("CLIENT_SECRET", "fa6d712219884fd28982562424fb8eda") # input your client_secret in between the quotation marks

client_credentials_manager = SpotifyClientCredentials(client_id = client_id, client_secret= client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
links = ["https://open.spotify.com/playlist/13eqT9yCaHVNcn1r14HqEo", "https://open.spotify.com/playlist/5AFYgjykqgPiDH6vLGnV3p", "https://open.spotify.com/playlist/77mJSFHVtzEmuxiYuTs5EP", "https://open.spotify.com/playlist/0Fug8hpuvWyMfLXWmmNKTO", "https://open.spotify.com/playlist/61l4S8GadpxWnb9ef6S0rY", "https://open.spotify.com/playlist/3rOn2Ro58lgbvf1iJQfIpH"]
output_files = ["rb.csv", "pop.csv", "party.csv", "kpop.csv", "cpop.csv", "study.csv"]

for link, output_file in zip(links, output_files):
    with open(output_file, "w", encoding = 'utf-8') as file:
            uris = link.split("/")[-1]
            writer = csv.writer(file)
            writer.writerow(['track_uri', 'track', 'artist', 'artist_popularity', 'followers', 'artist_genre', 'track_popularity', 'album'])
            for track in sp.playlist_tracks(uris)["items"]:
                #URI
                track_uri = track["track"]["uri"]

                #Track name
                track_name = track["track"]["name"]

                #Main Artist
                artist_uri = track["track"]["artists"][0]["uri"]
                artist_info = sp.artist(artist_uri)

                #Name, popularity, genre
                artist_name = track["track"]["artists"][0]["name"]
                artist_pop = artist_info["popularity"]
                artist_genre = artist_info["genres"]
                followers = artist_info["followers"]['total']

                #Album
                album = track["track"]["album"]["name"]

                #Popularity of the track
                track_pop = track["track"]["popularity"]

                # write a csv file
                writer.writerow([track_uri, track_name, artist_name, artist_pop, followers, artist_genre, track_pop, album])

In [4]:
rb = pd.read_csv("rb.csv")
pop = pd.read_csv("pop.csv")
party = pd.read_csv("party.csv")
kpop = pd.read_csv('kpop.csv')
cpop = pd.read_csv('cpop.csv')
study = pd.read_csv('study.csv')

rb.head(5)

,track_uri,track,artist,artist_popularity,followers,artist_genre,track_popularity,album
0,spotify:track:349Wc5mDu52d4Uv8Eg9WZv,Indigo,88rising,70,870947,"['asian american hip hop', 'pop']",63,Head In The Clouds II
1,spotify:track:50oEtTUNlce4TuZXQoJzXW,urs,NIKI,76,2221877,"['indonesian r&b', 'pop']",70,wanna take this downtown?
2,spotify:track:39rYdh4MkbRdoA7jvMUYYN,move!,NIKI,76,2221877,"['indonesian r&b', 'pop']",50,wanna take this downtown?
3,spotify:track:1NCUe90ZCiWTnWn3umrwqw,Vintage,NIKI,76,2221877,"['indonesian r&b', 'pop']",63,Zephyr
4,spotify:track:5TTXEcfsYLh6fTarLaevTi,lowkey,NIKI,76,2221877,"['indonesian r&b', 'pop']",79,lowkey


In [5]:
# Add playlist column to each table
rb['playlist'] = "r&b"
pop['playlist'] = "pop"
party['playlist'] = "party"
kpop['playlist'] = "kpop"
cpop['playlist'] = "cpop"
study['playlist'] = "study"

In [6]:
def uri_column(data):
    new = data['track_uri'].str.split(":", expand = True)
    data['track_uri'] = new[2].astype(str)
    
uri_column(rb)
uri_column(pop)
uri_column(party)
uri_column(kpop)
uri_column(cpop)
uri_column(study)

rb.head(5)

,track_uri,track,artist,artist_popularity,followers,artist_genre,track_popularity,album,playlist
0,349Wc5mDu52d4Uv8Eg9WZv,Indigo,88rising,70,870947,"['asian american hip hop', 'pop']",63,Head In The Clouds II,r&b
1,50oEtTUNlce4TuZXQoJzXW,urs,NIKI,76,2221877,"['indonesian r&b', 'pop']",70,wanna take this downtown?,r&b
2,39rYdh4MkbRdoA7jvMUYYN,move!,NIKI,76,2221877,"['indonesian r&b', 'pop']",50,wanna take this downtown?,r&b
3,1NCUe90ZCiWTnWn3umrwqw,Vintage,NIKI,76,2221877,"['indonesian r&b', 'pop']",63,Zephyr,r&b
4,5TTXEcfsYLh6fTarLaevTi,lowkey,NIKI,76,2221877,"['indonesian r&b', 'pop']",79,lowkey,r&b


In [7]:
def get_token():
    '''Get token'''
    auth_string = client_id + ":" + client_secret 
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), 'utf-8')

    url = "https://accounts.spotify.com/api/token"

    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded",
    }

    data = {'grant_type': "client_credentials"}
    result = post(url, headers = headers, data = data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

token = get_token()

headers = {'Authorization': 'Bearer {}'.format(token)}

In [8]:
base_url = 'https://api.spotify.com/v1/'
df_lists = [rb, pop, party, kpop, cpop, study]
output_files = ["trackrb.csv", "trackpop.csv", "trackparty.csv", "trackkpop.csv", "trackcpop.csv", "trackstudy.csv"]

for df, output_file in zip(df_lists, output_files):
    with open(output_file, "w", encoding = 'utf-8') as file:
        uris = df['track_uri'].to_list()
        features = {}
        writer = csv.writer(file)
        writer.writerow(["track_uri", "danceability", "valence", "energy", "tempo", "loudness", "speechiness", "instrumentalness", "liveness", "acousticness", "key"])
        for uri in uris:
            track_uri = uri
            
            features[uri] = {
                            'danceability': 0,
                            'valence': 0,
                            'energy': 0,
                            'tempo': 0,
                            'loudness': 0,
                            'speechiness': 0, 
                            'instrumentalness': 0,
                            'liveness': 0, 
                            'acousticness': 0, 
                            'key': 0
                            }

            s = requests.get(base_url + 'audio-features/' + uri, headers = headers)
            s = s.json()
            
            features[uri]['danceability'] = s['danceability']
            features[uri]['valence'] = s['valence']
            features[uri]['energy'] = s['energy']
            features[uri]['tempo'] = s['tempo']
            features[uri]['loudness'] = s['loudness']
            features[uri]['speechiness'] = s['speechiness']
            features[uri]['instrumentalness'] = s['instrumentalness']
            features[uri]['liveness'] = s['liveness']
            features[uri]['acousticness'] = s['acousticness']
            features[uri]['key'] = s['key']
            
            writer.writerow([track_uri, features[uri]['danceability'], features[uri]['valence'], features[uri]['energy'], features[uri]['tempo'], features[uri]['loudness'], features[uri]['speechiness'], 
                            features[uri]['instrumentalness'], features[uri]['liveness'], features[uri]['acousticness'], features[uri]['key']])

In [9]:
rb2 = pd.read_csv("trackrb.csv")
pop2 = pd.read_csv("trackpop.csv")
party2 = pd.read_csv("trackparty.csv")
kpop2 = pd.read_csv("trackkpop.csv")
cpop2 = pd.read_csv("trackcpop.csv")
study2 = pd.read_csv("trackstudy.csv")

rb2.head(5)

,track_uri,danceability,valence,energy,tempo,loudness,speechiness,instrumentalness,liveness,acousticness,key
0,349Wc5mDu52d4Uv8Eg9WZv,0.819,0.710,0.664,144.988,-5.612,0.0828,0.000002,0.2630,0.0958,7
1,50oEtTUNlce4TuZXQoJzXW,0.720,0.235,0.415,94.880,-9.527,0.3540,0.000001,0.0893,0.8340,0
2,39rYdh4MkbRdoA7jvMUYYN,0.829,0.801,0.490,126.922,-7.003,0.5080,0.000000,0.0861,0.5230,6
3,1NCUe90ZCiWTnWn3umrwqw,0.771,0.438,0.816,104.958,-7.104,0.0581,0.000963,0.3390,0.0114,2
4,5TTXEcfsYLh6fTarLaevTi,0.752,0.227,0.494,117.901,-8.180,0.0559,0.000000,0.0869,0.4320,0


In [10]:
rb = pd.merge(rb, rb2, how = 'inner', on = 'track_uri')
pop = pd.merge(pop, pop2, how = 'inner', on = 'track_uri')
party = pd.merge(party, party2, how = 'inner', on = 'track_uri')
kpop = pd.merge(kpop, kpop2, how = 'inner', on = 'track_uri')
cpop = pd.merge(cpop, cpop2, how = 'inner', on = 'track_uri')
study = pd.merge(study, study2, how = 'inner', on = 'track_uri')

In [11]:
rb.to_csv("rb.csv", index = False)
pop.to_csv("pop.csv", index = False)
party.to_csv("party.csv", index = False)
kpop.to_csv("kpop.csv", index = False)
cpop.to_csv("cpop.csv", index = False)
study.to_csv("study.csv", index = False)

In [12]:
df_complete = pd.concat([rb, pop, party, kpop, cpop, study], ignore_index = True)
df_complete.to_csv("df_complete.csv", index = False)

In [13]:
df_complete.head()

,track_uri,track,artist,artist_popularity,followers,artist_genre,track_popularity,album,playlist,danceability,valence,energy,tempo,loudness,speechiness,instrumentalness,liveness,acousticness,key
0,349Wc5mDu52d4Uv8Eg9WZv,Indigo,88rising,70,870947,"['asian american hip hop', 'pop']",63,Head In The Clouds II,r&b,0.819,0.710,0.664,144.988,-5.612,0.0828,0.000002,0.2630,0.0958,7
1,50oEtTUNlce4TuZXQoJzXW,urs,NIKI,76,2221877,"['indonesian r&b', 'pop']",70,wanna take this downtown?,r&b,0.720,0.235,0.415,94.880,-9.527,0.3540,0.000001,0.0893,0.8340,0
2,39rYdh4MkbRdoA7jvMUYYN,move!,NIKI,76,2221877,"['indonesian r&b', 'pop']",50,wanna take this downtown?,r&b,0.829,0.801,0.490,126.922,-7.003,0.5080,0.000000,0.0861,0.5230,6
3,1NCUe90ZCiWTnWn3umrwqw,Vintage,NIKI,76,2221877,"['indonesian r&b', 'pop']",63,Zephyr,r&b,0.771,0.438,0.816,104.958,-7.104,0.0581,0.000963,0.3390,0.0114,2
4,5TTXEcfsYLh6fTarLaevTi,lowkey,NIKI,76,2221877,"['indonesian r&b', 'pop']",79,lowkey,r&b,0.752,0.227,0.494,117.901,-8.180,0.0559,0.000000,0.0869,0.4320,0
